# EMBO Practical Course <br/>"Advanced methods in bioimage analysis"

***

Homepage: https://www.embl.org/about/info/course-and-conference-office/events/bia23-01/

***

## Day 2 - Session 1: Image Data Management - 10:15 to 11:15 "Get set"


<table style="table { position: relative;  display: inline-block; } img {  position: absolute;  left: 0;  right: 0;  width: auto;  height: 100%;  object-fit: cover;  object-position: center;}">
    <tr>
        <td style="vertical-align: top">
            <h3>Introduction</h3>
            <p>
                In this notebook, we'll look at issues arising from having many files
                (<b>and many file formats!</b>b>) on your file servers. A quick intro into working
                with POSIX file systems will be followed by tools for converting data into
                a common file-format: OME-TIFF.
            </p>
            <p>
                The two main goals are: (1) make sure you have the basics you might need
                for the rest of the course and (2) encourage you to explore these tools
                further on your own to speed up your day-to-day activities.
            </p>
            <p>
                Outline:
                <ol start="3">
                    <li>Bash & Scripting
                        <ol type="a">
                            <li>Bash basics</li>
                            <li>Our data directory</li>
                            <li>Extras: time permitting
                                <ul>
                                    <li>Permissions</li>
                                    <li>Simple scripting</li>
                                    <li>shasum</li>
                                    <li>h5tools</li>
                                </ul>
                            </li>
                        </ol>
                    </li>
                        <li>bftools
                        <ol type="a">
                            <li>showinf, bfconvert, etc.</li>
                            <li>Working with OME-TIFF</li>
                            <li>aicsimageio (time-permitting)</li>
                        </ol>
                    </li>
                </ol>
            </p>
        </td>
        <td>
            <center>
                <img src="images/falk/clara-shares-300dpi.png"/>
                <small>
                    <a href="https://github.com/zarr-developers/zarr-illustrations-falk-2022#clara-shares">"Clara shares"</a>
                    by Henning Falk, ©2022 NumFOCUS, is used under a CC BY 4.0 license. Modifications to this photo include cropping.
                </small>
            </center>
        </td>
    </tr>
</table>

### How to get started after the workshop?

[Binder](https://mybinder.org/) is a service for running notebooks online which can often be useful. In order to be be compatible with Binder, there is a folder `binder/` which contains the necessary files for configurig this repo. The `environment.yml` defines a [Conda](https://docs.conda.io/en/latest/) environment which you can install yourself.

In [1]:
ls binder/

apt.txt         environment.yml


In [2]:
cat binder/environment.yml

channels:
  - ome
  - conda-forge
  - joshmoore
  - defaults
dependencies:
  - python ~= 3.9.0
  - notebook
  - nb_conda_kernels
  - napari
  - napari-aicsimageio
  - bioformats_jar
  - ipython
  - wheel
  - requests
  - h5utils
  - s3fs
  - scikit-image
  - scipy
  - xarray
  - zarr >= 2.4.0
  - bftools
  - bioformats2raw
  - raw2ometiff
  - omero-py
  - go-mc
  - pv
  - pip
  - pip:
      - PyQT5~=5.13.1
      - ome-zarr
      - awscli
      - omero-cli-zarr
      - napari-ome-zarr
      - xmlschema
      - bash_kernel


You can perform those actions on your own system _without_ a Jupyter notebook. For that, you will need to install the correct tools like `bioformats2raw`. Instructions are available under https://github.com/ome/EMBO-Practical-Course-2023/blob/main/README.md
 
*** 

## 3a. Bash basics

If you are managing files on a server, it will often be easier to use the command-line once you learn it. And if you are using a command-line, it's likely going to be Bash. There are a large number of useful commands, and it will take time to become familiar with them all, <b>but it is worth your time.</b>

Often, the shorter the name the more useful it is. Let's take `ls` for "list". To learn more about a tool, you can use the `man` tool, or any many cases by passing the `-h` or `--help` argument.

In [3]:
man ls

LS(1)                       General Commands Manual                      LS(1)

NAME
     ls – list directory contents

SYNOPSIS
     ls [-@ABCFGHILOPRSTUWabcdefghiklmnopqrstuvwxy1%,] [--color=when]
        [-D format] [file ...]

DESCRIPTION
     For each operand that names a file of a type other than directory, ls
     displays its name as well as any requested, associated information.  For
     each operand that names a file of type directory, ls displays the names
     of files contained within that directory, as well as any requested,
     associated information.

     If no operands are given, the contents of the current directory are
     displayed.  If more than one operand is given, non-directory operands are
     displayed first; directory and non-directory operands are sorted
     separately and in lexicographical order.

     The following options are available:

     -@      Display extended attribute keys and sizes in long (-l) output.

     -A      Include directory entr

****

I bring up `ls` first because it can be used to find other commands:

In [4]:
ls -ltrad /bin /usr/bin/ /usr/local/bin

drwxr-xr-x   936 root     wheel  29952 Jul 11 10:56 /usr/bin/
drwxr-xr-x@   39 root     wheel   1248 Jul 11 10:56 /bin
drwxr-xr-x  2170 jamoore  admin  69440 Aug 28 22:23 /usr/local/bin


These are the directories where you will find many of them. For example, the command `bash` is one of them:

In [5]:
which bash

/bin/bash


`which` is a command that tells you where a command lives. `/bin/bash` is the command that gets run when you login. It's your "shell".

In [6]:
echo $SHELL

/bin/bash


So it's the context that everything you are doing is happening in:

In [7]:
pstree -p $$

-+= 00001 root /sbin/launchd
 \-+= 00560 jamoore /Applications/iTerm.app/Contents/MacOS/iTerm2
   \-+- 02144 jamoore /Users/jamoore/Library/Application Support/iTerm2/iTermSe
     \-+= 12058 root login -fp jamoore
       \-+= 12059 jamoore -bash
         \-+= 58533 jamoore /Users/jamoore/micromamba/envs/embo/bin/python3.9 /
           \-+= 95528 jamoore /Users/jamoore/micromamba/envs/embo/bin/python -m
             \-+= 95533 jamoore /bin/bash --rcfile /Users/jamoore/micromamba/en
               \-+= 95806 jamoore pstree -p 95533
                 \--- 95807 root ps -axwwo user,pid,ppid,pgid,command


That *shell* (or interpreter) holds the state of everything you are doing, including any variables you might set. Any time you start a new shell, you will need to make sure all of those variables are reset. For example, most of our notebooks will make use of the variable `YOURNAME`, e.g., to create directories, etc. You can find out your login name with the `whoami` command:

In [8]:
whoami

jamoore


In [9]:
YOURNAME=$(whoami)

In [10]:
echo $YOURNAME

jamoore


Now we have the variable we want set.

## 3c. Our data directory

Now with this out of the way, we can start to work with the data directories that are available on BAND.

`pwd` prints the directory that the current process is in:

In [11]:
pwd

/opt/EMBO-Workshop-2023


We want to move to another directory, for that we use the `cd` command:

In [12]:
cd /scratch/bioimagecourse2023/session1

Now we are in our working directory. Feel free to look around using `ls`. For example the flags `-ltra` mean: "show me a long listing of the files in reverse order by time and even show me the weird files starting with `.`".

In [13]:
ls -ltra

total 46024
-rw-r--r--   1 jamoore  wheel       529 Aug 23 18:13 2a.sh
-rw-r--r--   1 jamoore  wheel     42000 Aug 23 18:13 .2a.sh.un~
-rw-r--r--   1 jamoore  wheel        69 Aug 23 18:16 notes
-rw-r--r--   1 jamoore  wheel      3123 Aug 23 18:16 .notes.un~
-rw-r--r--   1 jamoore  wheel       469 Aug 23 18:50 2b.sh
-rw-r--r--   1 jamoore  wheel     27560 Aug 23 18:50 .2b.sh.un~
-rw-r--r--   1 jamoore  wheel       638 Aug 23 18:58 3a.sh
-rw-r--r--   1 jamoore  wheel     31925 Aug 23 18:58 .3a.sh.un~
drwxr-xr-x   3 jamoore  wheel        96 Aug 24 16:39 ..
drwxr-xr-x   4 jamoore  wheel       128 Aug 28 13:04 data
lrwxr-xr-x   1 jamoore  wheel        23 Aug 29 12:49 EMBO-Practical-Course-2023 -> /opt/EMBO-Workshop-2023
drwxr-xr-x  13 jamoore  wheel       416 Aug 29 12:50 .
-rw-r--r--   1 jamoore  wheel  23433941 Aug 29 12:50 OMEZarrReader-0.3.1-jar-with-dependencies.jar


There are many other flags you can use. "-S" flag means sort by size instead of time:

In [14]:
ls -lSr data/

total 0
-rw-r--r--  1 jamoore  wheel   0 Aug 23 18:01 a.fake
drwxr-xr-x  3 jamoore  wheel  96 Aug 23 18:31 cellprofiler


Another important tool for understanding the size of your data is `du`, for "disk usage". The `-sh` flags "say show me only a (s)ummary of the data and put it in (h)uman-readable form"

In [15]:
\du -sh data/

8.7M	data/


Now we want to make a directory for you to do all of your work in:
 * `mkdir -p`  means "create all parents" (but also: "don't fail if it also exists" which is useful for Jupyter notebooks!)
 * `test -e data ||` means "if the file or directory `data` does not exist, then create it by "linking" it into your own directory. This doesn't make a copy but does make it easier for you to find things.

In [16]:
mkdir -p /scratch/${YOURNAME}/session1
cd /scratch/${YOURNAME}/session1
test -e data || ln -s /scratch/bioimagecourse2023/session1/data data

`find` is a very powerful tool that we can use to explore directories full of lots of files. (In this case, we've added `tail` to only show the last time lines of all the output).

In [17]:
find . | tail -n 10

./a.ome.zarr/0/1/0/0/0
./a.ome.zarr/0/1/0/0/0/0
./a.ome.zarr/0/1/0/0/0/0/0
./a.ome.zarr/OME
./a.ome.zarr/OME/METADATA.ome.xml
./a.ome.zarr/OME/.zattrs
./a.ome.zarr/OME/.zgroup
./pos002.ome.tiff
./a.ome.tiff
./data


The `-name` flag can be used to match patterns on the filename. You can use `-iname` if you want to ignore the case of the files. The `wc` command counts words, lines, etc. The last number here tells us how many TIFF files were found.

In [18]:
find . -name "*.tiff" | wc

       2       2      31


You can also run a command on every file you found. The `file` command says what type of a file the operating system *assumes* a file to be. (It's not always right.)

In [19]:
find . -type f -exec file {} \;

./a.ome.zarr/.zattrs: JSON data
./a.ome.zarr/.zgroup: JSON data
./a.ome.zarr/0/.zattrs: JSON data
./a.ome.zarr/0/.zgroup: JSON data
./a.ome.zarr/0/0/.zarray: JSON data
./a.ome.zarr/0/0/0/0/0/0/0: Targa image data - Color (1-1024) 1604 x 65536 x 24 +2 ""
./a.ome.zarr/0/1/.zarray: JSON data
./a.ome.zarr/0/1/0/0/0/0/0: data
./a.ome.zarr/OME/METADATA.ome.xml: XML 1.0 document text, ASCII text, with very long lines (678), with no line terminators
./a.ome.zarr/OME/.zattrs: JSON data
./a.ome.zarr/OME/.zgroup: JSON data
./pos002.ome.tiff: TIFF image data, little-endian, direntries=17, height=1006, bps=8, compression=none, PhotometricIntepretation=RGB Palette, width=1000
./a.ome.tiff: TIFF image data, little-endian, direntries=16, height=512, bps=8, compression=none, PhotometricIntepretation=BlackIsZero, description=<?xml version="1.0" encoding="UTF-8"?><!-- Warning: this comment is an OME-XML metadata block, which contains crucial dimension, width=512


## Excercise 1:
### Use the tools above to find some data you want to use and symlink data into your own folder for later use.

In [35]:
## Do something here or on the command-line. If you are having issues, ask.

## 3d. Extras (time permitting)

### Permissions

In the directory listings above, e.g.:

```
drwxr-xr-x   936 root     wheel  29952 Jul 11 10:56 /usr/bin/
drwxr-xr-x@   39 root     wheel   1248 Jul 11 10:56 /bin
drwxr-xr-x  2086 jamoore  admin  66752 Aug 20 18:35 /usr/local/bin
```

the info here is **critical**:

```
PERMISSIONS ---- USER     GROUP   SIZE MODIFIED     NAME
```

Figuring out your user and group is pretty easy. `whoami` from above is your user. `id` can tell you your groups:

In [21]:
id

uid=501(jamoore) gid=20(staff) groups=20(staff),12(everyone),61(localaccounts),79(_appserverusr),80(admin),81(_appserveradm),98(_lpadmin),399(com.apple.access_ssh),33(_appstore),100(_lpoperator),204(_developer),250(_analyticsusers),395(com.apple.access_ftp),398(com.apple.access_screensharing),400(com.apple.access_remote_ae)


Together we can start to piece together whether or not you can read or edit a file. From https://www.grymoire.com/Unix/Permissions.html :

```
Three sections after the "d" for directory marker:
+------------+------+-------+
| Permission | Octal| Field |
+------------+------+-------+
| rwx------  | 700  | User  |
| ---rwx---  | 070  | Group |
| ------rwx  | 007  | Other |
+------------+------+-------+

Each section can be in one of 8 states:
+-----+---+--------------------------+
| rwx | 7 | Read, write and execute  |
| rw- | 6 | Read, write              |
| r-x | 5 | Read, and execute        |
| r-- | 4 | Read,                    |
| -wx | 3 | Write and execute        |
| -w- | 2 | Write                    |
| --x | 1 | Execute                  |
| --- | 0 | no permissions           |
+------------------------------------+

Examples:
+------------------------+-----------+
| chmod u=rwx,g=rwx,o=rx | chmod 775 | For world executables files
| chmod u=rwx,g=rx,o=    | chmod 750 | For executables by group only
| chmod u=rw,g=r,o=r     | chmod 644 | For world readable files
| chmod u=rw,g=r,o=      | chmod 640 | For group readable files
| chmod u=rw,go=         | chmod 600 | For private readable files
| chmod u=rwx,go=        | chmod 700 | For private executables
+------------------------+-----------+
```

This is *especially* important when using a shared folder like `/scratch`!

### Simple scripting

For loops can be fairly useful when working with many files:

In [22]:
for x in $(find . -name "*.fake");
do
    sha1sum ${x}
done

The other thing that can be very help if defining a function:

In [23]:
rename(){
  # I'm going to use this on every file
  # to change from .tiff to .tif.

  mv ${1} ${1%%.tiff}.tif

}

But this is obviously just a starting point. Perhaps check out a [Carpentry lesson](https://software-carpentry.org/lessons/) like [The Unix Shell](https://swcarpentry.github.io/shell-novice/) for more information. I can highly recommend them.

### shasum

In [24]:
for x in $(find . -type f);
do
    sha1sum ${x}
done

1a45bdb742869f84fd71ae0fd67f79f1b26923fe  ./a.ome.zarr/.zattrs
63de336a45370c236af207996ffd1bca2d7ae2f4  ./a.ome.zarr/.zgroup
74075dab6c0712c1d9ae80053ec1410e7f3099cf  ./a.ome.zarr/0/.zattrs
63de336a45370c236af207996ffd1bca2d7ae2f4  ./a.ome.zarr/0/.zgroup
965f764929f2da5e2691cb8049035975c2ccb5a0  ./a.ome.zarr/0/0/.zarray
7c2485fb91905ebd879e745d2e80f097584de271  ./a.ome.zarr/0/0/0/0/0/0/0
e6f199c7ed96964069fce5faa5e1cca8513a2320  ./a.ome.zarr/0/1/.zarray
dbc22354672642df9dcc2dcb5fffcced6a18b83a  ./a.ome.zarr/0/1/0/0/0/0/0
eda28681265146f6d153f53baaee09c030453123  ./a.ome.zarr/OME/METADATA.ome.xml
156e48269827cb4611d5a3899d862c60c8f483f4  ./a.ome.zarr/OME/.zattrs
63de336a45370c236af207996ffd1bca2d7ae2f4  ./a.ome.zarr/OME/.zgroup
722c2d9b3e9a7596389f03ffd5994be8be9c438f  ./pos002.ome.tiff
bd8850fd0b39f00ce78ccbfdacd1fdd2024100ff  ./a.ome.tiff


### h5tools

In [25]:
h5ls -h

usage: h5ls [OPTIONS] file[/OBJECT] [file[/[OBJECT]...]
  OPTIONS
   -h, -?, --help  Print a usage message and exit
   -a, --address   Print raw data address.  If dataset is contiguous, address
                   is offset in file of beginning of raw data. If chunked,
                   returned list of addresses indicates offset of each chunk.
                   Must be used with -v, --verbose option.
                   Provides no information for non-dataset objects.
   -d, --data      Print the values of datasets
   --enable-error-stack
                   Prints messages from the HDF5 error stack as they occur.
   --follow-symlinks
                   Follow symbolic links (soft links and external links)
                   to display target object information.
                   Without this option, h5ls identifies a symbolic link
                   as a soft link or external link and prints the value
                   assigned to the symbolic link; it does not provide any
         

<hr/>

## Half-time

<hr/>

In [26]:
##
## Setup & Sanity checks
##

YOURNAME=$(whoami)
WORKDIR=/scratch/${YOURNAME}/session1
test -e ${WORKDIR} || {
    echo Please run the steps above first.
    exit 1
}

In [27]:
cd /scratch/${YOURNAME}/session1

## 4a. bftools: showinf, bfconvert, etc.

bftools is a package of tools that can be downloaded from https://downloads.openmicroscopy.org/bio-formats/latest/artifacts/ or [conda](https://anaconda.org/ome/bftools) for working with bioimaging files from the command-line.

One of the tools, `formatlist` simply lists all the supported file formats with their associated file endings:

In [28]:
formatlist


File pattern: can read (pattern)
Zip: can read (zip)
Animated PNG: can read, can write, can write multiple (png)
JPEG: can read, can write (jpg, jpeg, jpe)
SlideBook 7 SLD (native): can read (sldy)
Portable Any Map: can read (pbm, pgm, ppm)
Flexible Image Transport System: can read (fits, fts)
PCX: can read (pcx)
Graphics Interchange Format: can read (gif)
Windows Bitmap: can read (bmp)
IPLab: can read (ipl)
IVision: can read (ipm)
RCPNL: can read (rcpnl)
Deltavision: can read (dv, r3d, r3d_d3d, dv.log, r3d.log)
Medical Research Council: can read (mrc, st, ali, map, rec, mrcs)
Gatan Digital Micrograph: can read (dm3, dm4)
Gatan DM2: can read (dm2)
Bitplane Imaris: can read (ims)
Openlab RAW: can read (raw)
OME-XML: can read, can write, can write multiple (ome, ome.xml)
Leica Image File Format: can read (lif)
Audio Video Interleave: can read, can write, can write multiple (avi)
PICT: can read (pict, pct)
SPCImage Data: can read (sdt)
SPC FIFO Data: can read (spc, set)
Encapsulated Post

We'll look at a few of these types.

In [29]:
showinf -nopix data/a.fake

Checking file format [Simulated data]
Initializing reader
FakeReader initializing data/a.fake
Initialization took 0.071s

Reading core metadata
filename = data/a.fake
Used files = [/System/Volumes/Data/scratch/jamoore/session1/data/a.fake]
Series count = 1
Series #0 :
	Image count = 1
	RGB = false (1) 
	Interleaved = false
	Indexed = false (true color)
	Width = 512
	Height = 512
	SizeZ = 1
	SizeT = 1
	SizeC = 1
	Tile size = 512 x 512
	Thumbnail size = 128 x 128
	Endianness = intel (little)
	Dimension order = XYZCT (certain)
	Pixel type = uint8
	Valid bits per pixel = 8
	Metadata complete = true
	Thumbnail series = false
	-----
	Plane #0 <=> Z 0, C 0, T 0


Reading global metadata

Reading metadata


In [30]:
showinf -nopix -omexml-only data/a.fake

<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
   <Image ID="Image:0" Name="a">
      <Pixels BigEndian="false" DimensionOrder="XYZCT" ID="Pixels:0" Interleaved="false" SignificantBits="8" SizeC="1" SizeT="1" SizeX="512" SizeY="512" SizeZ="1" Type="uint8">
         <Channel ID="Channel:0:0" SamplesPerPixel="1">
            <LightPath/>
         </Channel>
         <MetadataOnly/>
      </Pixels>
   </Image>
</OME>



In [31]:
test -e a.ome.tiff && rm -rf a.ome.tiff

In [32]:
bfconvert data/a.fake a.ome.tiff

data/a.fake
FakeReader initializing data/a.fake
[Simulated data] -> a.ome.tiff [OME-TIFF]
	Converted 1/1 planes (100%)
[done]
0.898s elapsed (18.0+154.0ms per plane, 698ms overhead)


In [33]:
showinf data/cellprofiler/fruit-fly-cells/POS218.pattern/POS218.pattern

Checking file format [File pattern]
Initializing reader
FilePatternReader initializing data/cellprofiler/fruit-fly-cells/POS218.pattern/POS218.pattern
TiffDelegateReader initializing /System/Volumes/Data/scratch/jamoore/session1/data/cellprofiler/fruit-fly-cells/POS218.pattern/01_POS218_Z00_T00_D.TIF
Reading IFDs
Populating metadata
Checking comment style
Populating OME metadata
TiffDelegateReader initializing /System/Volumes/Data/scratch/jamoore/session1/data/cellprofiler/fruit-fly-cells/POS218.pattern/01_POS218_Z00_T00_D.TIF
Reading IFDs
Populating metadata
Checking comment style
Populating OME metadata
TiffDelegateReader initializing /System/Volumes/Data/scratch/jamoore/session1/data/cellprofiler/fruit-fly-cells/POS218.pattern/01_POS218_Z00_T00_D.TIF
TiffDelegateReader initializing /System/Volumes/Data/scratch/jamoore/session1/data/cellprofiler/fruit-fly-cells/POS218.pattern/01_POS218_Z00_T00_D.TIF
TiffDelegateReader initializing /System/Volumes/Data/scratch/jamoore/session1/data/ce

In [34]:
showinf -nopix 'a&series=2.fake'

Initializing reader
FakeReader initializing a&series=2.fake
Initialization took 0.076s

Reading core metadata
filename = a&series=2.fake
Used files = [/System/Volumes/Data/scratch/jamoore/session1/a&series=2.fake]
Series count = 2
Series #0 :
	Image count = 1
	RGB = false (1) 
	Interleaved = false
	Indexed = false (true color)
	Width = 512
	Height = 512
	SizeZ = 1
	SizeT = 1
	SizeC = 1
	Tile size = 512 x 512
	Thumbnail size = 128 x 128
	Endianness = intel (little)
	Dimension order = XYZCT (certain)
	Pixel type = uint8
	Valid bits per pixel = 8
	Metadata complete = true
	Thumbnail series = false
	-----
	Plane #0 <=> Z 0, C 0, T 0

Series #1 :
	Image count = 1
	RGB = false (1) 
	Interleaved = false
	Indexed = false (true color)
	Width = 512
	Height = 512
	SizeZ = 1
	SizeT = 1
	SizeC = 1
	Tile size = 512 x 512
	Thumbnail size = 128 x 128
	Endianness = intel (little)
	Dimension order = XYZCT (certain)
	Pixel type = uint8
	Valid bits per pixel = 8
	Metadata complete = true
	Thumbnail series

You can see a graphical explanation of the "many-to-many" problem in https://downloads.openmicroscopy.org/presentations/2013/fs-workshop-paris/#/7/1 :

<table>
    <tr>
        <td>
            <img src="images/many-to-many/one-to-one.png"/>
        </td>
    </tr>
    <tr>
        <td>
            <img src="images/many-to-many/one-to-many.png"/>
        </td>
    </tr>
    <tr>
        <td>
            <img src="images/many-to-many/many-to-many.png"/>
        </td>
    </tr>
</table>


## Excercise 2:
### Use the tools above to find inspect the data you chose, and convert it to OME-TIFF.

In [36]:
## Do something here or on the command-line. If you are having issues, ask.

## License
Copyright (C) 2023 German BioImaging. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.